#### Загрузка библиотек

In [1]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import json
from pathlib import Path
import re
from copy import deepcopy
from tqdm import tqdm
import numpy as np

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
from matplotlib_venn import venn3, venn3_circles
from matplotlib_venn import venn2, venn2_circles

import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, GenerationConfig


In [2]:
MAX_TOKEN_COUNT = 2048
%env CUDA_VISIBLE_DEVICES=4
BOXED_PATTERN = r'\\boxed\{(.*?)\}'

env: CUDA_VISIBLE_DEVICES=4


### Подготовка данных

In [3]:
linal = {
    r"Decompose this vector $x\in X$ according to the basis $\{e_{i}\}_{i=1}{n}$ of \
the linear space $X$. \\ \
 \
\begin{equation*} \
x = \begin{pmatrix} 6 & 2 & 9 \end{pmatrix}^{\mathrm{T}}, \quad \
  e_{1} = \begin{pmatrix} 5 & 0 & 4 \end{pmatrix}^{\math{T}}, \quad \
  e_{2} = \begin{pmatrix} 5 & -1 & 0 \end{pmatrix}^{\mathrm{T}}, \quad \
  e_{3} = \begin{pmatrix} -1 & 0 & 4 \end{pmatrix}^{\mathrm{T}}. \
\end{equation*}" : r"[73/24, -2, -19/24]",

r"Find some basis for the space $X$ defined as a linear shell \
of vectors $\{x_{i}\}_{i=1}^{m}$. \
 \
\begin{gather*} \
  x_{1} = \begin{pmatrix} -1 & 4 & -3 & -2 \end{pmatrix}^{\mathrm{T}}, \quad \
  x_{2} = \begin{pmatrix} 3 & -7 & 5 & 3 \end{pmatrix}^{\mathrm{T}}, \\ \
  x_{3} = \begin{pmatrix} 3 & -2 & 1 & 0 \end{pmatrix}^{\mathrm{T}}, \quad \
  x_{4} = \begin{pmatrix} -4 & 1 & 0 & 1 \end{pmatrix}^{\mathrm{T}}. \
\end{gather*}" : r"\begin{gather*} \
        \begin{pmatrix} -1 & 4 & -3 & -2 \end{pmatrix}^{\mathrm{T}}, \quad \
        \begin{pmatrix} 0 & 5 & -4 & -3 \end{pmatrix}^{\mathrm{T}}, \quad \
        \begin{pmatrix}  0 & 0 & -4 & 9 \end{pmatrix}^{\mathrm{T}}. \
    \end{gather*}",


r"Find the basis of the intersection of subspaces $L_{1}$ and $L_{2}$ of a linear \
space $X$ spanned by the corresponding vector systems $\{x_{i}\}_{i=1}^{p}$ and \
$\{y_{j}\}_{j=1}^{q}$ respectively. \
 \
\begin{gather*} \
  L_{1} = \left\langle (1,2,3,1,1)^\mathrm{T}, (1,0,1,-2,-2)^\mathrm{T}, \
    (2,0,1,-1,0)^\mathrm{T},(0,1,1,0,0)^\mathrm{T} \right\rangle, \\ \
  L_{2} = \left\langle (1,2,0,0,2)^\mathrm{T}, (0,1,-2,3,-3)^\mathrm{T}, \
    (-1,2,1,2,0)^\mathrm{T}, (1,1,-2,0,0)^\mathrm{T} \right\rangle. \
\end{gather*}" : r"$(0,0,1,0,0)^\mathrm{T}$, $(0,72,0,43,0)^\mathrm{T}$, $(18,-0,0,55,-76)^\mathrm{T}$.",

r"Find the coefficients of the linear form $f\in X^{\ast}$ in a given basis \
$\{e_{i}\}_{i=1}{n}$ spaces $X$. \
 \
\begin{gather*} \
f(x) = 4 \xi^{1} + 2 \xi^{2} + \xi^{3}, \\ \
e_{1} =\begin{pmatrix} 5 & 0 & 4 \end{pmatrix}^{\math{T}}, \quad \
  e_{2} = \begin{pmatrix} 5 & -1 & 0 \end{pmatrix}^{\mathrm{T}}, \quad \
  e_{3} = \begin{pmatrix} -1 & 0 & 4 \end{pmatrix}^{\mathrm{T}}. \
\end{gather*}" : r"(24, 18, 0)",

r"Find the basis of a linear shell $L$ given by a system of linear \
algebraic equations. \
 \
\begin{gather*} \
  \left\{ \begin{array}{lcr} \
    x_1 - x_2 + x_3 - x_4 &=& 0; \\ \
    x_1 + x_2 + 2x_3 + 3x_4 &=& 0;\\ \
    2x_1 + 4x_2 + 5x_3 + 10x_4 &=& 0;\\ \
    2x_1 - 4x_2 + x_3 - 6x_4 &=& 0; \
  \end{array} \right. \
\end{gather*}": r"$(3, 1, -2, 0)^\mathrm{T}$, $(5, 0, -4, 1)^\mathrm{T}$", 

r"Find the coordinates of the vector $x$ of the linear space $X$ in the basis \
$\{\$\{\tilde e_{j}\}_{j=1}^{n}$ if its coordinates in the basis are known \
$\{e_{i}\}_{i=1}^{n}$. \
 \
\begin{gather*} \
  x = \begin{pmatrix} 6 & 2 & 9 \end{pmatrix}^{\mathrm{T}}, \\ \
  \tilde e_{1} = 5e_{1} + 4e_{3}, \quad \
  \tilde e_{2} = 5e_{1} - e_{2} , \quad \
  \tilde e_{3} = -e_{1} + 4e_{3}. \
\end{gather*}" : r"$(\frac{73}{24}, -2, -\frac{19}{24})^\mathrm{T}$"
}
linal_df = []
for k, v in linal.items():
    task = {}
    task["subfield"] = "GeolinLinal"
    task["question"] = k
    task["final_answer"] =  "['$" + v + "$']"
    task["solution"] = None
    task["solution_length"] = None
    linal_df.append(task)

linal_df = pd.DataFrame(linal_df)
linal_df.index = np.arange(0, len(linal_df))

linal_df.to_csv("/home/anmilka/different/linal_df.csv")

linal_dict = linal_df.T.to_dict().values()
print(len(linal_dict))


6


In [4]:
linal_df.head()

,subfield,question,final_answer,solution,solution_length
0,GeolinLinal,Decompose this vector $x\in X$ according to th...,"['$[73/24, -2, -19/24]$']",None,None
1,GeolinLinal,Find some basis for the space $X$ defined as a...,['$\begin{gather*} \\n \begin{pmatrix} ...,None,None
2,GeolinLinal,Find the basis of the intersection of subspace...,"['$$(0,0,1,0,0)^\mathrm{T}$, $(0,72,0,43,0)^\m...",None,None
3,GeolinLinal,Find the coefficients of the linear form $f\in...,"['$(24, 18, 0)$']",None,None
4,GeolinLinal,Find the basis of a linear shell $L$ given by ...,"['$$(3, 1, -2, 0)^\mathrm{T}$, $(5, 0, -4, 1)^...",None,None


In [5]:
def find_boxed_content(text):
    start_index = text.rfind(r'\boxed{')
    if start_index == -1:
        return None
    start_index += len(r'\boxed{')    
    stack = ['{']
    content = []

    for i in range(start_index, len(text)):
        char = text[i]
        content.append(char)
        if char == '{':
            stack.append(char)
        elif char == '}':
            if stack:
                stack.pop()
            if not stack:
                return ''.join(content[:-1])
    return None

In [ ]:
task_6_1 = r"Find the coordinates of the vector $x$ of the linear space $X$ in the basis \
$\{\$\{\tilde e_{j}\}_{j=1}^{n}$ if its coordinates in the basis are known \
$\{e_{i}\}_{i=1}^{n}$. \
 \
\begin{gather*} \
  x = \begin{pmatrix} 6 & 2 & 9 \end{pmatrix}^{\mathrm{T}}, \\ \
  \tilde e_{1} = 5e_{1} + 4e_{3}, \quad \
  \tilde e_{2} = 5e_{1} - e_{2} , \quad \
  \tilde e_{3} = -e_{1} + 4e_{3}. \
\end{gather*}"

example_6_1 = "import Mathlib.Data.Matrix.Basic \n\
import Aesop \n\
import Mathlib.Tactic.NormNum \n\
import Mathlib.Tactic.FinCases \n\
    \n\
import Mathlib.Tactic.Ring \n\
import Mathlib.Algebra.BigOperators.Fin \n\
import Mathlib.Tactic.RewriteSearch \n\
import AutoSolver \n\
 \n\
 \n\
variable (e1 e2 e3: ℚ) \n\
 \n\
def x: ℚ := 6 * e1 + 2 * e2 + 9 * e3 \n\
 \n\
 \n\
@[simp] def e1': ℚ := 5 * e1 + 4 * e3 \n\
@[simp] def e2': ℚ := 5 * e1 - e2 \n\
@[simp] def e3': ℚ := - e1 + 4 * e3 \n\
 \n\
variable (x1 x2 x3 : ℚ) \n\
 \
@[simp] axiom x_axiom: x e1 e2 e3 = x1 * e1' e1 e3 + x2 * e2' e1 e2 + x3 * e3' e1 e3 \n\
 \n\
@[simp] def transformation_matrix: Matrix (Fin 3) (Fin 3) ℚ := Matrix.of ![![5, 5, -1], ![0, -1, 0], ![4, 0, 4]] \n\
@[simp] def transformation_matrix_rev : Matrix (Fin 3) (Fin 3) ℚ := Matrix.of ![![1/6, 5/6, 1/24], ![0, -1, 0], ![-1/6, -5/6, 5/24]] \n\
 \n\
@[simp] theorem transformation_matrix_theorem: ![e1' e1 e3, e2' e1 e2, e3' e1 e3] = Matrix.vecMul ![e1, e2, e3] transformation_matrix := by \n\
  auto_solver \n\
 \n\
 \n\
@[simp] theorem matrix_rev: transformation_matrix * transformation_matrix_rev = Matrix.of ![![1, 0, 0], ![0, 1, 0], ![0, 0, 1]] := by \n\
  auto_solver \n\
 \n\
 \n\
@[simp] theorem x': Matrix.mulVec transformation_matrix_rev ![6, 2, 9] = ![73/24, -2, -19/24] := by \n\
  auto_solver"

print(example_6_1)
print(task_6_1)

### Lean-Star-Plus

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

# question_template = "<|im_start|>user\nMy LEAN 4 state is:\n```{state}```\nPlease write down the reasoning that leads to the possible next tactic and then predict the tactic to help me prove the theorem.<|im_end|>\n<|im_start|>assistant\n"

model_name = "ScalableMath/Lean-STaR-plus"
model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.bfloat16, device_map="auto", trust_remote_code=True)

tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)

# state = "x : \u211d\nn : \u2115\nh\u2080 : -1 < x\nh\u2081 : 0 < n\n\u22a2 1 + \u2191n * x \u2264 (1 + x) ^ n"
# question = question_template.format(state=state)

question = f"Here is an example of task and LEAN4 code for solving it. Task: {task_6_1} LEAN4 code: {example_6_1}.\
Please, write LEAN4 code for the following task:" +  r"Decompose this vector $x\in X$ according to the basis $\{e_{i}\}_{i=1}{n}$ of \
the linear space $X$. \\ \
 \
\begin{equation*} \
x = \begin{pmatrix} 6 & 2 & 9 \end{pmatrix}^{\mathrm{T}}, \quad \
  e_{1} = \begin{pmatrix} 5 & 0 & 4 \end{pmatrix}^{\math{T}}, \quad \
  e_{2} = \begin{pmatrix} 5 & -1 & 0 \end{pmatrix}^{\mathrm{T}}, \quad \
  e_{3} = \begin{pmatrix} -1 & 0 & 4 \end{pmatrix}^{\mathrm{T}}. \
\end{equation*}"

input_tensor = torch.tensor([tokenizer.encode(question)])
outputs = model.generate(input_tensor.to(model.device), max_new_tokens=MAX_TOKEN_COUNT)

result = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(result)

In [8]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

# question_template = "<|im_start|>user\nMy LEAN 4 state is:\n```{state}```\nPlease write down the reasoning that leads to the possible next tactic and then predict the tactic to help me prove the theorem.<|im_end|>\n<|im_start|>assistant\n"

model_name = "ScalableMath/Lean-STaR-plus"
model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.bfloat16, device_map="auto", trust_remote_code=True)
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)


for _, task in tqdm(enumerate(linal_dict)):
    question = task['question'] + '\n\nPlease, reason step-by-step and write the valid Lean 4 code to solve the task.'

    input_tensor = torch.tensor([tokenizer.encode(question)])
    outputs = model.generate(input_tensor.to(model.device), max_new_tokens=MAX_TOKEN_COUNT)

    detailed_result = tokenizer.decode(outputs[0], skip_special_tokens=True)
    print(_)
    print(detailed_result)
    print('\n'*5)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

0it [00:00, ?it/s]The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
1it [02:49, 169.99s/it]

0
Decompose this vector $x\in X$ according to the basis $\{e_{i}\}_{i=1}{n}$ of \
the linear space $X$. \\ \
 \
\begin{equation*} \
x = \begin{pmatrix} 6 & 2 & 9 \end{pmatrix}^{\mathrm{T}}, \quad \
  e_{1} = \begin{pmatrix} 5 & 0 & 4 \end{pmatrix}^{\math{T}}, \quad \
  e_{2} = \begin{pmatrix} 5 & -1 & 0 \end{pmatrix}^{\mathrm{T}}, \quad \
  e_{3} = \begin{pmatrix} -1 & 0 & 4 \end{pmatrix}^{\mathrm{T}}. \
\end{equation*}

Please, reason step-by-step and write the valid Lean 4 code to solve the task. 

Here is the step-by-step reasoning:

1. We need to express the vector $x$ as a linear combination of the basis vectors $e_{1}, e_{2}, e_{3}$.
2. We can set up a system of linear equations to find the coefficients $a_{1}, a_{2}, a_{3}$ such that $x = a_{1}e_{1} + a_{2}e_{2} + a_{3}e_{3}$.
3. By solving this system, we can find the values of $a_{1}, a_{2}, a_{3}$.

Here is the corresponding Lean 4 code:
```lean
import Lean

namespace Ex1

def e₁ : Vector ℝ 3 := ⟨5, 0, 4⟩
def e₂ : Vector ℝ 3 

2it [05:38, 168.91s/it]

1
Find some basis for the space $X$ defined as a linear shell \
of vectors $\{x_{i}\}_{i=1}^{m}$. \
 \
\begin{gather*} \
  x_{1} = \begin{pmatrix} -1 & 4 & -3 & -2 \end{pmatrix}^{\mathrm{T}}, \quad \
  x_{2} = \begin{pmatrix} 3 & -7 & 5 & 3 \end{pmatrix}^{\mathrm{T}}, \\ \
  x_{3} = \begin{pmatrix} 3 & -2 & 1 & 0 \end{pmatrix}^{\mathrm{T}}, \quad \
  x_{4} = \begin{pmatrix} -4 & 1 & 0 & 1 \end{pmatrix}^{\mathrm{T}}. \
\end{gather*}

Please, reason step-by-step and write the valid Lean 4 code to solve the task. 

Here is the proof in LEAN 4:```lean
import Mathlib.Data.Matrix.Notation
import Mathlib.LinearAlgebra.Matrix.ToLin

variable {R₁ : Type} [CommRing R₁]

#check Matrix.toLin' (A₁ := R₁) (n := 4) (m := 4)
```

```lean
import Mathlib.Data.Matrix.Notation
import Mathlib.LinearAlgebra.Matrix.ToLin

variable {R₁ : Type} [CommRing R₁]

#check Matrix.toLin' (A₁ := R₁) (n := 4) (m := 4)
```

```lean
import Mathlib.Data.Matrix.Notation
import Mathlib.LinearAlgebra.Matrix.ToLin

variable {R

...

3
Find the coefficients of the linear form $f\in X^{\ast}$ in a given basis \
$\{e_{i}\}_{i=1}{n}$ spaces $X$. \
 \
\begin{gather*} \
f(x) = 4 \xi^{1} + 2 \xi^{2} + \xi^{3}, \\ \
e_{1} =\begin{pmatrix} 5 & 0 & 4 \end{pmatrix}^{\math{T}}, \quad \
  e_{2} = \begin{pmatrix} 5 & -1 & 0 \end{pmatrix}^{\mathrm{T}}, \quad \
  e_{3} = \begin{pmatrix} -1 & 0 & 4 \end{pmatrix}^{\mathrm{T}}. \
\end{gather*}

Please, reason step-by-step and write the valid Lean 4 code to solve the task. 

Here is the step-by-step reasoning:

1. We need to find the coefficients of the linear form $f$ in the given basis $\{e_{i}\}_{i=1}^{n}$. 
2. The coefficients can be found by evaluating $f$ at each basis vector $e_{i}$ and then solving the resulting system of linear equations.
3. We can use the `solve_for` tactic to solve the system of linear equations.

Here is the Lean 4 code to solve the task: 

```lean
import Lean

namespace LinearMap

variable {R : Type} {M : Type} {N : Type} {ι : Type} [CommRing R] [AddComm

5it [14:19, 173.05s/it]

4
Find the basis of a linear shell $L$ given by a system of linear \
algebraic equations. \
 \
\begin{gather*} \
  \left\{ \begin{array}{lcr} \
    x_1 - x_2 + x_3 - x_4 &=& 0; \\ \
    x_1 + x_2 + 2x_3 + 3x_4 &=& 0;\\ \
    2x_1 + 4x_2 + 5x_3 + 10x_4 &=& 0;\\ \
    2x_1 - 4x_2 + x_3 - 6x_4 &=& 0; \
  \end{array} \right. \
\end{gather*}

Please, reason step-by-step and write the valid Lean 4 code to solve the task. 

Here is the step-by-step reasoning:

1. We need to find the basis of the linear shell $L$ given by the system of linear algebraic equations. 
2. We can use the `linear_combination` tactic to express the system of equations in a more manageable form.
3. By simplifying the system, we can find the basis of the linear shell $L$.

Here is the Lean 4 code to solve the task: 

```lean
linear_combination (norm := ring1) a * (1, -1, 1, -1) + b * (1, 1, 2, 3) + c * (2, 4, 5, 10) + d * (2, -4, 1, -6)
```

# Explanation

The `linear_combination` tactic is used to express the system of

6it [17:18, 173.07s/it]

5
Find the coordinates of the vector $x$ of the linear space $X$ in the basis \
$\{\$\{\tilde e_{j}\}_{j=1}^{n}$ if its coordinates in the basis are known \
$\{e_{i}\}_{i=1}^{n}$. \
 \
\begin{gather*} \
  x = \begin{pmatrix} 6 & 2 & 9 \end{pmatrix}^{\mathrm{T}}, \\ \
  \tilde e_{1} = 5e_{1} + 4e_{3}, \quad \
  \tilde e_{2} = 5e_{1} - e_{2} , \quad \
  \tilde e_{3} = -e_{1} + 4e_{3}. \
\end{gather*}

Please, reason step-by-step and write the valid Lean 4 code to solve the task. 

Here is the proof in LEAN 4:```lean
import Lean

namespace LinearMap

variable {R : Type} [CommRing R] {M : Type} [AddCommGroup M] [Module R M]

def toMatrix' (f : M →ₗ[R] M) {ι : Type} [Fintype ι] (b : Basis ι R M) : Matrix ι ι R :=
  Matrix.of (fun i => f.toMatrix b (b i))

theorem toMatrix'_apply (f : M →ₗ[R] M) {ι : Type} [Fintype ι] (b : Basis ι R M) (i j : ι) :
  f.toMatrix' b i j = f (b j) i := by simp [toMatrix']

end LinearMap

namespace Matrix

variable {R : Type} [CommRing R] {n : Type} [Fintype n] [

### DeepSeek-Prover-V1.5-RL

In [ ]:
from vllm import LLM, SamplingParams

model_name = "deepseek-ai/DeepSeek-Prover-V1.5-RL"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = LLM(model=model_name, max_num_batched_tokens=8192, seed=1, trust_remote_code=True, dtype='half')

device = "cuda"
model.to(device)


question = f"Here is an example of task and LEAN4 code for solving it. Task: {task_6_1} LEAN4 code: {example_6_1}.\
Please, write LEAN4 code for the following task:" +  r"Decompose this vector $x\in X$ according to the basis $\{e_{i}\}_{i=1}{n}$ of \
the linear space $X$. \\ \
 \
\begin{equation*} \
x = \begin{pmatrix} 6 & 2 & 9 \end{pmatrix}^{\mathrm{T}}, \quad \
  e_{1} = \begin{pmatrix} 5 & 0 & 4 \end{pmatrix}^{\math{T}}, \quad \
  e_{2} = \begin{pmatrix} 5 & -1 & 0 \end{pmatrix}^{\mathrm{T}}, \quad \
  e_{3} = \begin{pmatrix} -1 & 0 & 4 \end{pmatrix}^{\mathrm{T}}. \
\end{equation*}"

model_inputs = question

sampling_params = SamplingParams(
    temperature=1.0,
    max_tokens=MAX_TOKEN_COUNT,
    top_p=0.95,
    n=1,
)
#model_inputs = [prompt + code_prefix]
model_outputs = model.generate(
    model_inputs,
    sampling_params,
    use_tqdm=False,
)
result = model_inputs + model_outputs[0].outputs[0].text
print(result)

In [5]:
from vllm import LLM, SamplingParams

model_name = "deepseek-ai/DeepSeek-Prover-V1.5-RL"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = LLM(model=model_name, max_num_batched_tokens=8192, seed=1, trust_remote_code=True, dtype='half')

# device = "cuda"
# model.to(device)

for _, task in tqdm(enumerate(linal_dict)):
    question = task['question'] + '\n\nPlease, reason step-by-step and write the valid Lean 4 code to solve the task.'

    model_inputs = question

    sampling_params = SamplingParams(
        temperature=1.0,
        max_tokens=MAX_TOKEN_COUNT,
        top_p=0.95,
        n=1,
    )
    #model_inputs = [prompt + code_prefix]
    model_outputs = model.generate(
        model_inputs,
        sampling_params,
        use_tqdm=False,
    )
    detailed_result = model_outputs[0].outputs[0].text
    print(_)
    print(detailed_result)
    print('\n'*5)

2025-04-20 15:09:16.431763: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-04-20 15:09:16.431834: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-04-20 15:09:16.434037: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-04-20 15:09:16.447009: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-04-20 15:09:17.938498: W tensorflow/compiler/tf2

WARNING 04-20 15:09:23 config.py:1865] Casting torch.bfloat16 to torch.float16.
INFO 04-20 15:09:36 config.py:350] This model supports multiple tasks: {'generate', 'embedding'}. Defaulting to 'generate'.
INFO 04-20 15:09:36 llm_engine.py:249] Initializing an LLM engine (v0.6.4.post1) with config: model='deepseek-ai/DeepSeek-Prover-V1.5-RL', speculative_config=None, tokenizer='deepseek-ai/DeepSeek-Prover-V1.5-RL', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, tokenizer_revision=None, trust_remote_code=True, dtype=torch.float16, max_seq_len=4096, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto, quantization_param_path=None, device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='outlines'), observability_config=ObservabilityConfig(otlp_traces_endpoint=None, collect_model_forward_tim

Loading safetensors checkpoint shards:   0% Completed | 0/2 [00:00<?, ?it/s]


INFO 04-20 15:09:47 model_runner.py:1077] Loading model weights took 12.8725 GB
INFO 04-20 15:09:49 worker.py:232] Memory profiling results: total_gpu_memory=31.73GiB initial_memory_usage=13.23GiB peak_torch_memory=13.87GiB memory_usage_post_profile=13.26GiB non_torch_memory=0.38GiB kv_cache_size=14.31GiB gpu_memory_utilization=0.90
INFO 04-20 15:09:49 gpu_executor.py:113] # GPU blocks: 1953, # CPU blocks: 546
INFO 04-20 15:09:49 gpu_executor.py:117] Maximum concurrency for 4096 tokens per request: 7.63x
INFO 04-20 15:09:55 model_runner.py:1400] Capturing cudagraphs for decoding. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI.
INFO 04-20 15:09:55 model_runner.py:1404] If out-of-memory error occurs during cudagraph capture, consider decreasing `gpu_memory_utilization` or switching to eager mode. You can also reduce the `max_num_seqs` as needed to decrease memory usage.
INFO

1it [00:27, 27.65s/it]

...

3
To find the coefficients of the linear form \( f \in X^* \) in the given basis \(\{e_i\}_{i=1}^n\) spaces \(X\), we need to express \( f \) in terms of the basis vectors. The linear form \( f \) is given by:

\[ f(x) = 4 \xi^1 + 2 \xi^2 + \xi^3 \]

The basis vectors are:

\[ e_1 = \begin{pmatrix} 5 \\ 0 \\ 4 \end{pmatrix}, \quad e_2 = \begin{pmatrix} 5 \\ -1 \\ 0 \end{pmatrix}, \quad e_3 = \begin{pmatrix} -1 \\ 0 \\ 4 \end{pmatrix} \]

To find the coefficients, we need to express \( f \) as a linear combination of the basis vectors. This means we need to find scalars \( a_1, a_2, a_3 \) such that:

\[ f = a_1 e_1 + a_2 e_2 + a_3 e_3 \]

We can set up a system of equations by evaluating \( f \) at each basis vector:

1. For \( e_1 \):
\[ f(e_1) = 4 \cdot 5 + 2 \cdot 0 + 1 \cdot 4 = 20 + 0 + 4 = 24 \]

2. For \( e_2 \):
\[ f(e_2) = 4 \cdot 5 + 2 \cdot (-1) + 1 \cdot 0 = 20 - 2 + 0 = 18 \]

3. For \( e_3 \):
\[ f(e_3) = 4 \cdot (-1) + 2 \cdot 0 + 1 \cdot 4 = -4 + 0 + 4 = 0 \]

This give

5it [02:27, 28.25s/it]

4
To find the basis of the linear shell \( L \) given by the system of linear algebraic equations, we need to determine the dimension of the null space of the matrix formed by the system. Here is the step-by-step process to solve the problem:

1. **Formulate the Matrix Representation**:
   The given system of equations can be written as:
   \[
   \begin{cases}
   x_1 - x_2 + x_3 - x_4 = 0 \\
   x_1 + x_2 + 2x_3 + 3x_4 = 0 \\
   2x_1 + 4x_2 + 5x_3 + 10x_4 = 0 \\
   2x_1 - 4x_2 + x_3 - 6x_4 = 0
   \end{cases}
   \]
   This can be represented as:
   \[
   A \mathbf{x} = \mathbf{0}
   \]
   where \( A \) is the coefficient matrix:
   \[
   A = \begin{pmatrix}
   1 & -1 & 1 & -1 \\
   1 & 1 & 2 & 3 \\
   2 & 4 & 5 & 10 \\
   2 & -4 & 1 & -6
   \end{pmatrix}
   \]

2. **Reduce the Matrix to Row Echelon Form**:
   To find the basis of the null space, we need to reduce the matrix \( A \) to its row echelon form. Performing Gaussian elimination, we get:
   \[
   A = \begin{pmatrix}
   1 & -1 & 

6it [02:50, 28.42s/it]

5
To find the coordinates of the vector \( x \) in the basis \(\{\tilde{e}_{j}\}_{j=1}^{n}\) given its coordinates in the basis \(\{e_{i}\}_{i=1}^{n}\), we need to perform the following steps:

1. **Understand the given information:**
   - The vector \( x \) has coordinates \([6, 2, 9]\) in the basis \(\{e_{i}\}_{i=1}^{n}\).
   - The basis vectors \(\tilde{e}_{j}\) are expressed in terms of the basis vectors \(e_{i}\):
     \[
     \tilde{e}_{1} = 5e_{1} + 4e_{3}, \quad \tilde{e}_{2} = 5e_{1} - e_{2}, \quad \tilde{e}_{3} = -e_{1} + 4e_{3}.
     \]

2. **Express \( x \) in terms of the basis \(\{e_{i}\}_{i=1}^{n}\):**
   \[
   x = 6e_{1} + 2e_{2} + 9e_{3}.
   \]

3. **Find the coordinates of \( x \) in the basis \(\{\tilde{e}_{j}\}_{j=1}^{n}\):**
   - We need to express \( x \) in terms of \(\tilde{e}_{1}\), \(\tilde{e}_{2}\), and \(\tilde{e}_{3}\).
   - Substitute the expressions for \(\tilde{e}_{j}\) in terms of \(e_{i}\):
     \[
     x = 6e_{1} + 2e_{2} + 9e_{3}.
     \]
     Substi

### internlm2-math-plus-7b

In [ ]:
# Load model directly
from transformers import AutoModelForCausalLM
model_name = "internlm/internlm2-math-plus-7b"

model = AutoModelForCausalLM.from_pretrained(model_name, trust_remote_code=True)
tokenizer = AutoTokenizer.from_pretrained(model_name, use_Fast=False, trust_remote_code=True)

device = "cuda"
model.to(device)

question = f"Here is an example of task and LEAN4 code for solving it. Task: {task_6_1} LEAN4 code: {example_6_1}.\
Please, write LEAN4 code for the following task:" +  r"Decompose this vector $x\in X$ according to the basis $\{e_{i}\}_{i=1}{n}$ of \
the linear space $X$. \\ \
 \
\begin{equation*} \
x = \begin{pmatrix} 6 & 2 & 9 \end{pmatrix}^{\mathrm{T}}, \quad \
  e_{1} = \begin{pmatrix} 5 & 0 & 4 \end{pmatrix}^{\math{T}}, \quad \
  e_{2} = \begin{pmatrix} 5 & -1 & 0 \end{pmatrix}^{\mathrm{T}}, \quad \
  e_{3} = \begin{pmatrix} -1 & 0 & 4 \end{pmatrix}^{\mathrm{T}}. \
\end{equation*}"

messages = [
    {"role": "user", "content": question}
]
input_tensor = tokenizer.apply_chat_template(messages, add_generation_prompt=True, return_tensors="pt")
outputs = model.generate(input_tensor.to(model.device), max_new_tokens=MAX_TOKEN_COUNT)
detailed_result = tokenizer.decode(outputs[0][input_tensor.shape[1]:], skip_special_tokens=True)

print(detailed_result)

In [6]:
from transformers import AutoModelForCausalLM
model_name = "internlm/internlm2-math-plus-7b"

model = AutoModelForCausalLM.from_pretrained(model_name, trust_remote_code=True)
tokenizer = AutoTokenizer.from_pretrained(model_name, use_Fast=False, trust_remote_code=True)

device = "cuda"
model.to(device)

for _, task in tqdm(enumerate(linal_dict)):
    question = task['question'] + '\n\nPlease, reason step-by-step and write the valid Lean 4 code to solve the task.'

    messages = [
        {"role": "user", "content": question}
    ]
    input_tensor = tokenizer.apply_chat_template(messages, add_generation_prompt=True, return_tensors="pt")
    outputs = model.generate(input_tensor.to(model.device), max_new_tokens=MAX_TOKEN_COUNT)
    detailed_result = tokenizer.decode(outputs[0][input_tensor.shape[1]:], skip_special_tokens=True)
    print(_)
    print(detailed_result)
    print('\n'*5)

2025-04-20 13:52:41.903969: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-04-20 13:52:41.904030: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-04-20 13:52:41.906378: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-04-20 13:52:41.918852: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-04-20 13:52:43.323378: W tensorflow/compiler/tf2

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

0it [00:00, ?it/s]The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
1it [03:22, 202.94s/it]

0
To decompose the vector $x$ according to the basis $\{e_{i}\}_{i=1}^{n}$ of the linear space $X$, we need to find the coefficients $a_{i}$ such that $x = \sum_{i=1}^{n} a_{i} e_{i}$. This can be done by solving the system of linear equations given by the components of the vectors.

Let's start by setting up the system of equations using the components of the vectors $x$, $e_{1}$, $e_{2}$, and $e_{3}$.

We have:

$$
\begin{align*}
6 &= 5a_{1} + 5a_{2} - a_{3}, \\
2 &= -a_{1} + 0a_{2} + 0a_{3}, \\
9 &= 4a_{1} + 0a_{2} + 4a_{3}.
\end{align*}
$$

We can solve this system using the `solve` function from the `LinearAlgebra` module in Lean.
```lean
namespace Ex

variable {X : Type _} [NormedAddCommGroup X] [NormedSpace ℝ X]
variable {n : ℕ} (e : Fin n → X) (x : X)

noncomputable def decompose (x : X) : ℝ ^ n :=
  ∑ i : Fin n, (e i ⟪e i, x⟫_ℝ) / ∥e i∥ ^ 2

end Ex

```

```lean
namespace Ex

variable {X : Type _} [NormedAddCommGroup X] [NormedSpace ℝ X]
variable {n : ℕ} (e : Fin n → X) (x : X

2it [06:42, 201.03s/it]

...

3
To find the coefficients of the linear form $f$ in the given basis $\{e_{i}\}_{i=1}^{n}$, we need to express $f$ as a linear combination of the basis vectors. This means we want to find scalars $a_{1}, a_{2}, a_{3}$ such that:

$$f = a_{1}e_{1} + a_{2}e_{2} + a_{3}e_{3}$$

Given $f(x) = 4 \xi^{1} + 2 \xi^{2} + \xi^{3}$, we can set up the following equation:

$$4 \xi^{1} + 2 \xi^{2} + \xi^{3} = a_{1}\begin{pmatrix} 5 & 0 & 4 \end{pmatrix}^{\mathrm{T}} + a_{2}\begin{pmatrix} 5 & -1 & 0 \end{pmatrix}^{\mathrm{T}} + a_{3}\begin{pmatrix} -1 & 0 & 4 \end{pmatrix}^{\mathrm{T}}$$

This equation can be rewritten in terms of a matrix equation:

$$\begin{pmatrix} 4 \\ 2 \\ 1 \end{pmatrix} = \begin{pmatrix} 5 & 5 & -1 \\ 0 & -1 & 0 \\ 4 & 0 & 4 \end{pmatrix} \begin{pmatrix} a_{1} \\ a_{2} \\ a_{3} \end{pmatrix}$$

We can solve this system of equations using the `solve` function from the `LinearAlgebra` module in Lean. Let's write the code to find the coefficients $a_{1}, a_{2}, a_{3}$.
```lean
i

5it [16:55, 204.11s/it]

4
To find the basis of the linear shell $L$ given by the system of linear algebraic equations, we first need to express the system in matrix form and then solve for the basis vectors.

The given system of equations is:
\[
\begin{gather*}
\left\{
\begin{array}{lcr}
x_1 - x_2 + x_3 - x_4 &=& 0; \\
x_1 + x_2 + 2x_3 + 3x_4 &=& 0;\\
2x_1 + 4x_2 + 5x_3 + 10x_4 &=& 0;\\
2x_1 - 4x_2 + x_3 - 6x_4 &=& 0; \\
\end{array}
\right.
\end{gather*}
\]

We can represent this system as a matrix equation $Ax = 0$, where $A$ is the matrix of coefficients and $x$ is the vector of variables. Let's define the matrix $A$ and solve for the basis of the null space of $A$.
```lean
import Lean
import Mathlib.Data.Matrix.Notation
import Mathlib.LinearAlgebra.Matrix.NonsingularInverse
import Mathlib.LinearAlgebra.Matrix.FiniteDimensional
import Mathlib.LinearAlgebra.Matrix.Basis

variable {R : Type} [Field R]

#check Matrix
#check Matrix.transpose
#check Matrix.mulVec
#check Matrix.mul
#check Matrix.nonsing_inv
#chec

6it [20:17, 202.85s/it]

5
We can use the `change_of_basis` function to find the coordinates of the vector $x$ in the basis $\{\tilde e_{j}\}_{j=1}^{n}$. The function takes the coordinates of $x$ in the basis $\{e_{i}\}_{i=1}^{n}$ and the change of basis matrix as arguments. The change of basis matrix is formed by the coordinates of the basis vectors $\{\tilde e_{j}\}_{j=1}^{n}$ in the basis $\{e_{i}\}_{i=1}^{n}$. 

In this case, the change of basis matrix is:

$$
\begin{pmatrix}
5 & 0 & -1 \\
4 & -1 & 0 \\
0 & 4 & 4
\end{pmatrix}
$$

We can compute the coordinates of $x$ in the basis $\{\tilde e_{j}\}_{j=1}^{n}$ by multiplying the coordinates of $x$ in the basis $\{e_{i}\}_{i=1}^{n}$ with the change of basis matrix. 

Let's compute this in Lean.
```lean
import Mathlib.LinearAlgebra.Matrix.NonsingularInverse

variable {n : ℕ} (e : Fin n → Matrix (Fin 1) (Fin 1) ℝ) (x : Matrix (Fin 1) (Fin 3) ℝ)

def change_of_basis (e : Fin n → Matrix (Fin 1) (Fin 1) ℝ) (f : Fin n → Matrix (Fin 1) (Fin 1) ℝ) :=
 Matrix.of fun 

In [5]:
from transformers import AutoModelForCausalLM
model_name = "internlm/internlm2-math-plus-7b"

model = AutoModelForCausalLM.from_pretrained(model_name, trust_remote_code=True)
tokenizer = AutoTokenizer.from_pretrained(model_name, use_Fast=False, trust_remote_code=True)

device = "cuda"
model.to(device)

for _, task in tqdm(enumerate(linal_dict)):
    question = task['question'] + '\n\nSolve this via Lean 4.'

    messages = [
        {"role": "user", "content": question}
    ]
    input_tensor = tokenizer.apply_chat_template(messages, add_generation_prompt=True, return_tensors="pt")
    outputs = model.generate(input_tensor.to(model.device), max_new_tokens=MAX_TOKEN_COUNT)
    detailed_result = tokenizer.decode(outputs[0][input_tensor.shape[1]:], skip_special_tokens=True)
    print(_)
    print(detailed_result)
    print('\n'*5)

2025-04-20 14:22:54.629451: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-04-20 14:22:54.629510: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-04-20 14:22:54.631761: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-04-20 14:22:54.644296: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-04-20 14:22:56.113916: W tensorflow/compiler/tf2

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

0it [00:00, ?it/s]The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
1it [03:11, 191.69s/it]

0
# Proof in Natural langauge

To decompose the vector $x$ into its components along the basis $\{e_{i}\}_{i=1}^{n}$, we need to find the coefficients $a_{i}$ such that $x = \sum_{i=1}^{n} a_{i} e_{i}$. We can do this by solving the system of linear equations given by the components of $x$ and the basis vectors.

The system of equations is:
\begin{align*}
6 &= 5a_{1} + 5a_{2} - a_{3}, \\
2 &= -a_{2}, \\
9 &= 4a_{1} + 0a_{2} + 4a_{3}.
\end{align*}

From the second equation, we have $a_{2} = -2$. Substituting this into the first and third equations, we get:
\begin{align*}
6 &= 5a_{1} - 10 - a_{3}, \\
9 &= 4a_{1} + 4a_{3}.
\end{align*}

Simplifying these equations, we have:
\begin{align*}
5a_{1} - a_{3} &= 16, \\
4a_{1} + 4a_{3} &= 9.
\end{align*}

Multiplying the second equation by 5, we get:
\begin{align*}
5a_{1} - a_{3} &= 16, \\
20a_{1} + 20a_{3} &= 45.
\end{align*}

Subtracting the first equation from the second, we have:
\begin{align*}
19a_{1} + 21a_{3} &= 29.
\end{align*}

Dividing

2it [06:24, 192.24s/it]

...

3
# Proof in Natural langauge

To find the coefficients of the linear form $f$ in the given basis $\{e_{i}\}_{i=1}^{n}$, we need to express $f$ as a linear combination of the basis vectors. We can do this by setting up a system of equations where the coefficients of the basis vectors are the unknowns. The system of equations is given by:

$$
\begin{align*}
f(e_{1}) &= 4 \cdot 5 + 2 \cdot 0 + 1 \cdot 4 = 20 + 0 + 4 = 24, \\
f(e_{2}) &= 4 \cdot 5 + 2 \cdot (-1) + 1 \cdot 0 = 20 - 2 + 0 = 18, \\
f(e_{3}) &= 4 \cdot (-1) + 2 \cdot 0 + 1 \cdot 4 = -4 + 0 + 4 = 0.
\end{align*}
$$

Solving this system of equations, we find that the coefficients are $a_{1} = 24$, $a_{2} = 18$, and $a_{3} = 0$. Therefore, the coefficients of the linear form $f$ in the given basis are $24$, $18$, and $0$.

# Proof in LEAN 4

```lean
conv_lhs => rw [← LinearMap.sum_comp]
```

# Explanation

The proof uses the `conv_lhs => rw [← LinearMap.sum_comp]` tactic to rewrite the left-hand side of the goal. This tactic app

5it [15:58, 191.64s/it]

4
```lean
import Lean

namespace LinearShell

def R := Int

def x1 x2 x3 x4 : R := 0

#check x1

end LinearShell
```

```lean
import Lean

namespace LinearShell

def R := Int

def x1 x2 x3 x4 : R := 0

#check x1

end LinearShell
```

```lean
import Lean

namespace LinearShell

def R := Int

def x1 x2 x3 x4 : R := 0

#check x1

end LinearShell
```

```lean
import Lean

namespace LinearShell

def R := Int

def x1 x2 x3 x4 : R := 0

#check x1

end LinearShell
```

```lean
import Lean

namespace LinearShell

def R := Int

def x1 x2 x3 x4 : R := 0

#check x1

end LinearShell
```

```lean
import Lean

namespace LinearShell

def R := Int

def x1 x2 x3 x4 : R := 0

#check x1

end LinearShell
```

```lean
import Lean

namespace LinearShell

def R := Int

def x1 x2 x3 x4 : R := 0

#check x1

end LinearShell
```

```lean
import Lean

namespace LinearShell

def R := Int

def x1 x2 x3 x4 : R := 0

#check x1

end LinearShell
```

```lean
import Lean

namespace LinearShell

def R := Int

def x1 x2 x3

6it [19:09, 191.50s/it]

5
```lean
import Mathbin.LinearAlgebra.Matrix.Basis

variable {R : Type*} [CommRing R] {n : ℕ} (hn : 0 < n)

variable {K V : Type*} [Field K] [AddCommGroup V] [Module K V]

variable {B : Fin n → V} (hB : Basis (Fin n) K V)

#align aligns

theorem aligns (i : Fin n) : ∃ g : (Fin n → K) →ₗ[K] K, g (B i) = 1 := by
 refine' ⟨(B i).coeff ∘ B.repr, _⟩
 ext j
 simp only [Function.comp_apply, Basis.repr_self, Finsupp.coeLinearMap, coe_repr,
  Finsupp.single_apply, eq_comm]
 rw [if_pos (Fin.ext_iff.mp _)]
 rfl
```

```lean
import Mathbin.LinearAlgebra.Matrix.Basis

variable {R : Type*} [CommRing R] {n : ℕ} (hn : 0 < n)

variable {K V : Type*} [Field K] [AddCommGroup V] [Module K V]

variable {B : Fin n → V} (hB : Basis (Fin n) K V)

#align aligns

theorem aligns (i : Fin n) : ∃ g : (Fin n → K) →ₗ[K] K, g (B i) = 1 := by
 refine' ⟨(B i).coeff ∘ B.repr, _⟩
 ext j
 simp only [Function.comp_apply, Basis.repr_self, Finsupp.coeLinearMap, coe_repr,
  Finsupp.single_apply, eq_comm]
 rw [if_pos (Fin.ext

In [5]:
from transformers import AutoModelForCausalLM
model_name = "internlm/internlm2-math-plus-7b"

model = AutoModelForCausalLM.from_pretrained(model_name, trust_remote_code=True)
tokenizer = AutoTokenizer.from_pretrained(model_name, use_Fast=False, trust_remote_code=True)

device = "cuda"
model.to(device)

question = r"Decompose this vector $x\in X$ according to the basis $\{e_{i}\}_{i=1}{n}$ of \
the linear space $X$. \\ \
 \
\begin{equation*} \
x = \begin{pmatrix} 6 & 2 & 9 \end{pmatrix}^{\mathrm{T}}, \quad \
  e_{1} = \begin{pmatrix} 5 & 0 & 4 \end{pmatrix}^{\math{T}}, \quad \
  e_{2} = \begin{pmatrix} 5 & -1 & 0 \end{pmatrix}^{\mathrm{T}}, \quad \
  e_{3} = \begin{pmatrix} -1 & 0 & 4 \end{pmatrix}^{\mathrm{T}}. \
\end{equation*}"


cot = r"To decompose the vector \( x \in X \) according to the basis \( \{e_{i}\}_{i=1}^{n} \) of the linear space \( X \), we need to express \( x \) as a linear combination of the basis vectors \( e_1, e_2, \) and \( e_3 \). \
This means we need to find scalars \( a_1, a_2, \) and \( a_3 \) such that: \[ x = a_1 e_1 + a_2 e_2 + a_3 e_3 \] \
Given: \[ x = \begin{pmatrix} 6 & 2 & 9 \end{pmatrix}^{\mathrm{T}}, \quad e_{1} = \begin{pmatrix} 5 & 0 & 4 \end{pmatrix}^{\mathrm{T}}, \quad e_{2} = \begin{pmatrix} 5 & -1 & 0 \end{pmatrix}^{\mathrm{T}}, \quad e_{3} = \begin{pmatrix} -1 & 0 & 4 \end{pmatrix}^{\mathrm{T}} \] \
We can write this equation in matrix form: \[ \begin{pmatrix} 6 & 2 & 9 \end{pmatrix}^{\mathrm{T}} = a_1 \begin{pmatrix} 5 & 0 & 4 \end{pmatrix}^{\mathrm{T}} + a_2 \begin{pmatrix} 5 & -1 & 0 \end{pmatrix}^{\mathrm{T}} + a_3 \begin{pmatrix} -1 & 0 & 4 \end{pmatrix}^{\mathrm{T}} \] \
This gives us a system of linear equations: \[ \begin{cases} 5a_1 + 5a_2 - a_3 = 6 \\ 0a_1 - a_2 + 0a_3 = 2 \\ 4a_1 + 0a_2 + 4a_3 = 9 \end{cases} \] \
We can simplify this system: \
1. \( 5a_1 + 5a_2 - a_3 = 6 \) \
2. \( -a_2 = 2 \) \
3. \( 4a_1 + 4a_3 = 9 \) \
From the second equation, we solve for \( a_2 \): \[ a_2 = -2 \] \
Substitute \( a_2 = -2 \) into the first equation: \[ 5a_1 + 5(-2) - a_3 = 6 \] \[ 5a_1 - 10 - a_3 = 6 \] \[ 5a_1 - a_3 = 16 \quad \text{(Equation 4)} \] \
Substitute \( a_2 = -2 \) into the third equation: \[ 4a_1 + 4a_3 = 9 \] \[ a_1 + a_3 = \frac{9}{4} \quad \text{(Equation 5)} \] \
Now we have a system of two equations with two unknowns: \[ \begin{cases} 5a_1 - a_3 = 16 \\ a_1 + a_3 = \frac{9}{4} \end{cases} \] \
Add these two equations to eliminate \( a_3 \): \[ (5a_1 - a_3) + (a_1 + a_3) = 16 + \frac{9}{4} \] \[ 6a_1 = 16 + \frac{9}{4} \] \[ 6a_1 = \frac{64}{4} + \frac{9}{4} \] \[ 6a_1 = \frac{73}{4} \] \[ a_1 = \frac{73}{24} \] \
Substitute \( a_1 = \frac{73}{24} \) back into Equation 5: \[ \frac{73}{24} + a_3 = \frac{9}{4} \] \[ a_3 = \frac{9}{4} - \frac{73}{24} \] \[ a_3 = \frac{54}{24} - \frac{73}{24} \] \[ a_3 = -\frac{19}{24} \] \
Thus, the scalars \( a_1, a_2, \) and \( a_3 \) are: \[ a_1 = \frac{73}{24}, \quad a_2 = -2, \quad a_3 = -\frac{19}{24} \] \
Therefore, the decomposition of \( x \) according to the basis \( \{e_{i}\}_{i=1}^{n} \) is: \[ \boxed{\frac{73}{24} e_1 - 2 e_2 - \frac{19}{24} e_3} \]"
question = question + '\n\n' + cot + '\n\nConvert this answer into Lean4.'

messages = [
    {"role": "user", "content": question}
]
input_tensor = tokenizer.apply_chat_template(messages, add_generation_prompt=True, return_tensors="pt")
outputs = model.generate(input_tensor.to(model.device), max_new_tokens=MAX_TOKEN_COUNT)
detailed_result = tokenizer.decode(outputs[0][input_tensor.shape[1]:], skip_special_tokens=True)
print(_)
print(detailed_result)
print('\n'*5)

config.json:   0%|          | 0.00/858 [00:00<?, ?B/s]

configuration_internlm2.py:   0%|          | 0.00/8.84k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/internlm/internlm2-math-plus-7b:
- configuration_internlm2.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


modeling_internlm2.py:   0%|          | 0.00/81.0k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/internlm/internlm2-math-plus-7b:
- modeling_internlm2.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
2025-04-20 15:26:19.171786: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-04-20 15:26:19.171865: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-04-20 15:26:19.174587: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-04-20 15:26:19.188202: I tensorflow/core/platform/cpu_feature_guard.cc:18

...

A new version of the following files was downloaded from https://huggingface.co/internlm/internlm2-math-plus-7b:
- tokenization_internlm2.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
A new version of the following files was downloaded from https://huggingface.co/internlm/internlm2-math-plus-7b:
- tokenization_internlm2_fast.py
- tokenization_internlm2.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


tokenizer.model:   0%|          | 0.00/1.48M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/95.0 [00:00<?, ?B/s]

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


      subfield                                           question  \
0  GeolinLinal  Decompose this vector $x\in X$ according to th...   
1  GeolinLinal  Find some basis for the space $X$ defined as a...   
2  GeolinLinal  Find the basis of the intersection of subspace...   
3  GeolinLinal  Find the coefficients of the linear form $f\in...   
4  GeolinLinal  Find the basis of a linear shell $L$ given by ...   

                                        final_answer solution solution_length  
0                          ['$[73/24, -2, -19/24]$']     None            None  
1  ['$\begin{gather*} \\n        \begin{pmatrix} ...     None            None  
2  ['$$(0,0,1,0,0)^\mathrm{T}$, $(0,72,0,43,0)^\m...     None            None  
3                                  ['$(24, 18, 0)$']     None            None  
4  ['$$(3, 1, -2, 0)^\mathrm{T}$, $(5, 0, -4, 1)^...     None            None  
Here is the formal statement in Lean4:```lean
-- Let X be a linear space over the field ℝ with basis {e_i

### internlm2-step-prover

In [ ]:
# Load model directly
from transformers import AutoModelForCausalLM
model_name = "internlm/internlm2-step-prover"

model = AutoModelForCausalLM.from_pretrained(model_name, trust_remote_code=True)
tokenizer = AutoTokenizer.from_pretrained(model_name, use_Fast=False, trust_remote_code=True)

device = "cuda"
model.to(device)

question = f"Here is an example of task and LEAN4 code for solving it. Task: {task_6_1} LEAN4 code: {example_6_1}.\
Please, write LEAN4 code for the following task:" +  r"Decompose this vector $x\in X$ according to the basis $\{e_{i}\}_{i=1}{n}$ of \
the linear space $X$. \\ \
 \
\begin{equation*} \
x = \begin{pmatrix} 6 & 2 & 9 \end{pmatrix}^{\mathrm{T}}, \quad \
  e_{1} = \begin{pmatrix} 5 & 0 & 4 \end{pmatrix}^{\math{T}}, \quad \
  e_{2} = \begin{pmatrix} 5 & -1 & 0 \end{pmatrix}^{\mathrm{T}}, \quad \
  e_{3} = \begin{pmatrix} -1 & 0 & 4 \end{pmatrix}^{\mathrm{T}}. \
\end{equation*}"

messages = [
    {"role": "user", "content": question}
]
input_tensor = tokenizer.apply_chat_template(messages, add_generation_prompt=True, return_tensors="pt")
outputs = model.generate(input_tensor.to(model.device), max_new_tokens=MAX_TOKEN_COUNT)
detailed_result = tokenizer.decode(outputs[0][input_tensor.shape[1]:], skip_special_tokens=True)

print(detailed_result)

In [ ]:
from transformers import AutoModelForCausalLM
model_name = "internlm/internlm2-step-prover"

model = AutoModelForCausalLM.from_pretrained(model_name, trust_remote_code=True)
tokenizer = AutoTokenizer.from_pretrained(model_name, use_Fast=False, trust_remote_code=True)

device = "cuda"
model.to(device)

for _, task in tqdm(enumerate(linal_dict)):
    question = task['question'] + '\n\nPlease, reason step-by-step and write the valid Lean 4 code to solve the task.'

    messages = [
        {"role": "user", "content": question}
    ]
    input_tensor = tokenizer.apply_chat_template(messages, add_generation_prompt=True, return_tensors="pt")
    outputs = model.generate(input_tensor.to(model.device), max_new_tokens=MAX_TOKEN_COUNT)
    detailed_result = tokenizer.decode(outputs[0][input_tensor.shape[1]:], skip_special_tokens=True)
    print(_)
    print(detailed_result)
    print('\n'*5)

In [5]:
from transformers import AutoModelForCausalLM
model_name = "internlm/internlm2-step-prover"

model = AutoModelForCausalLM.from_pretrained(model_name, trust_remote_code=True)
tokenizer = AutoTokenizer.from_pretrained(model_name, use_Fast=False, trust_remote_code=True)

device = "cuda"
model.to(device)

for _, task in tqdm(enumerate(linal_dict)):
    question = task['question'] + '\n\nSolve this via Lean 4.'

    messages = [
        {"role": "user", "content": question}
    ]
    input_tensor = tokenizer.apply_chat_template(messages, add_generation_prompt=True, return_tensors="pt")
    outputs = model.generate(input_tensor.to(model.device), max_new_tokens=MAX_TOKEN_COUNT)
    detailed_result = tokenizer.decode(outputs[0][input_tensor.shape[1]:], skip_special_tokens=True)
    print(_)
    print(detailed_result)
    print('\n'*5)

config.json:   0%|          | 0.00/947 [00:00<?, ?B/s]

configuration_internlm2.py:   0%|          | 0.00/7.02k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/internlm/internlm2-step-prover:
- configuration_internlm2.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


modeling_internlm2.py:   0%|          | 0.00/60.0k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/internlm/internlm2-step-prover:
- modeling_internlm2.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
2025-04-20 14:43:35.489531: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-04-20 14:43:35.489591: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-04-20 14:43:35.491975: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-04-20 14:43:35.505345: I tensorflow/core/platform/cpu_feature_guard.cc:182

...

3
PROOFSTEP simp [LinearForm, LinearMap, Function.comp, Matrix.mulVec, Matrix.dotProduct]
 roasted
 roasted
 roasted
้








5it [12:35, 147.95s/it]

4
PROOFSTEP simp only [add_comm, add_left_comm, add_assoc, sub_eq_add_neg] at * 









6it [14:27, 144.63s/it]

5
PROOFSTEP simp [Matrix.transpose, Matrix.mulVec, Matrix.dotProduct, Matrix.mul_apply, Matrix.transpose_apply]
 roasted
 roasted
 roasted
้






